<a href="https://colab.research.google.com/github/abhinav-TB/text-summarization/blob/main/experimenting_text_clustering_using_SOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Pre-Processing


In [ ]:
#installing hugging face dataset library
!pip install datasets

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 53.0 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 134 kB 59.5 MB/s 
     |████████████████████████████████| 1.1 MB 66.6 MB/s 
     |████████████████████████████████| 127 kB 42.3 MB/s 
     |████████████████████████████████| 144 kB 70.3 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 73.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# imports
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from datasets import load_dataset

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#Loading dataset
dataset = load_dataset('ccdv/cnn_dailymail', '3.0.0')

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#splitting train and test sets
df_train = dataset['train']
df_test = dataset['test']['article']
print(len(df_train))
print(len(df_test))

287113
11490


In [ ]:
#sample item in dataset
text = df_train[0]['article']
text

'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he want

### Tokenization
In this step, the text is split into smaller units. We can use either sentence tokenization or word tokenization based on our problem statement

In [ ]:
## Sentence Tokensing
from nltk.tokenize import sent_tokenize

text_tokens = sent_tokenize(text)
org_text = text_tokens.copy()
print(len(text_tokens))
text_tokens[0]

76


"It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria."

### Puncuation Removal
In this step, all the punctuations from the text are removed. string library of Python contains some pre-defined list of punctuations such as ‘!”#$%&'()*+,-./:;?@[\]^_`{|}~’
}~’

In [ ]:
# Removing pucuations
import string

def remove_punctuation(text):
    return "".join([i for  i in text if i not in string.punctuation])

for i in range(len(text_tokens)):
    text_tokens[i] = remove_punctuation(text_tokens[i])


text_tokens[0]

'Its official US President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria'

### Stop word Removal
Stopwords are the commonly used words and are removed from the text as they do not add any value to the analysis. These words carry less or no meaning.

In [ ]:
# removing stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

new_text_tokens = []

for sentence in text_tokens:
    temp = " ".join([w for w in sentence.split() if not w.lower() in stop_words])
    new_text_tokens.append(temp)

text_tokens = new_text_tokens
text_tokens[0]

'official US President Barack Obama wants lawmakers weigh whether use military force Syria'

### Lemmatization
It stems the word but makes sure that it does not lose its meaning.  Lemmatization has a pre-defined dictionary that stores the context of words and checks the word in the dictionary while diminishing.


In [ ]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return "".join(lemm_text)

for i in range(len(text_tokens)):
    text_tokens[i] = lemmatize(text_tokens[i])


In [ ]:
text_tokens

['official US President Barack Obama wants lawmakers weigh whether use military force Syria',
 'Obama sent letter heads House Senate Saturday night hours announcing believes military action Syrian targets right step take alleged use chemical weapons',
 'proposed legislation Obama asks Congress approve use military force deter disrupt prevent degrade potential future uses chemical weapons weapons mass destruction',
 'step set turn international crisis fierce domestic political battle',
 'key questions looming debate UN weapons inspectors find Syria',
 'happens Congress votes',
 'Syrian government react',
 'televised address White House Rose Garden earlier Saturday president said would take case Congress wants',
 'believe authority carry military action without specific congressional authorization know country stronger take course actions even effective said',
 'debate issues big business usual',
 'Obama said top congressional leaders agreed schedule debate body returns Washington Septem

## mappings

In [ ]:
# text_tokens mapping to org sentence
org_mapping = {}
for i ,sentence in enumerate(text_tokens):
    org_mapping[sentence] = org_text[i]

# positional mapping for sentence order

pos_mapping={}
for i , sentence in enumerate(org_text):
    pos_mapping[sentence] = i


## Text Scoring

In [ ]:
# TF IDF score
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords   

def _create_frequency_matrix(sentences):
    """
    accepting sentences as inputs
    creating a mapping between a sentence and freq of words in that sentence
    """
    frequency_matrix = {}
    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent] = freq_table

    return frequency_matrix

def _create_tf_matrix(freq_matrix):
    """
    term frequency matrix - the number of times a word occur in a sentence / total number of words in that sentence
    """
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

def _create_documents_per_words(freq_matrix):
    """
    finds freqency of the words in the entire corpus
    """
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table
 
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    """
    creating the idf matrix with the direct log formula
    """
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    """
    Element wise multiplication of tf and idf matrix to get the tf-idf matrix
    """
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

total_documents = len(text_tokens)
freq_matrix = _create_frequency_matrix(text_tokens)

tf_matrix = _create_tf_matrix(freq_matrix)

count_doc_per_words = _create_documents_per_words(freq_matrix)

idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)

tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)

sentence_scores = _score_sentences(tf_idf_matrix)
print(sentence_scores)
feature1 = sentence_scores


{'official US President Barack Obama wants lawmakers weigh whether use military force Syria': 0.088677423714, 'Obama sent letter heads House Senate Saturday night hours announcing believes military action Syrian targets right step take alleged use chemical weapons': 0.062355738907764136, 'proposed legislation Obama asks Congress approve use military force deter disrupt prevent degrade potential future uses chemical weapons weapons mass destruction': 0.07843545119930559, 'step set turn international crisis fierce domestic political battle': 0.18296435013431178, 'key questions looming debate UN weapons inspectors find Syria': 0.14116674181828323, 'happens Congress votes': 0.5265945322056034, 'Syrian government react': 0.42936550293985887, 'televised address White House Rose Garden earlier Saturday president said would take case Congress wants': 0.08649882962550873, 'believe authority carry military action without specific congressional authorization know country stronger take course acti

In [ ]:
# SENTENCE POSITION
from math import cos
tt = dict()
thresh = len(text_tokens) * 0.2
min = thresh * len(text_tokens)
max = thresh * 2 * len(text_tokens)
for i in range(0,len(text_tokens)):
  if ( i == 0 or i == (len(text_tokens) - 1) ):
    tt[text_tokens[i]] = 2
  else:
    tt[text_tokens[i]] = float(cos((i - min) / ((1 / max) - min )))

feature2 = tt
print(feature2)

{'official US President Barack Obama wants lawmakers weigh whether use military force Syria': 2, 'Obama sent letter heads House Senate Saturday night hours announcing believes military action Syrian targets right step take alleged use chemical weapons': 0.5410302086819669, 'proposed legislation Obama asks Congress approve use military force deter disrupt prevent degrade potential future uses chemical weapons weapons mass destruction': 0.5417580213526336, 'step set turn international crisis fierce domestic political battle': 0.5424854280557806, 'key questions looming debate UN weapons inspectors find Syria': 0.5432124282463243, 'happens Congress votes': 0.5439390213794856, 'Syrian government react': 0.5446652069107903, 'televised address White House Rose Garden earlier Saturday president said would take case Congress wants': 0.5453909842960699, 'believe authority carry military action without specific congressional authorization know country stronger take course actions even effective s

In [ ]:
# NOUN AND NUMERAL COUNTS
from nltk.tokenize import sent_tokenize
from collections import Counter
nltk.download('averaged_perceptron_tagger')

sentence = ''
# initializing weights for nouns and numerals
nounwt = 0.3 
numwt = 0.7

for t in text_tokens:
  sentence = sentence + t + '. '

tokens = sent_tokenize(sentence)

noun_stuff = dict()
for t in tokens:
  tok2 = nltk.word_tokenize(t)
  t2 = nltk.Text(tok2)
  tags = nltk.pos_tag(t2)
  counts = Counter(tag for word, tag in tags if tag == 'NN' or tag == 'CD')
  if 'NN' in counts and 'CD' in counts:
    noun_stuff[t[:-1]] = counts['NN']*nounwt + counts['CD']*numwt
  elif 'NN' in counts and 'CD' not in counts:
    noun_stuff[t[:-1]] = counts['NN']*nounwt
  elif 'NN' not in counts and 'CD' in counts:
    noun_stuff[t[:-1]] = counts['CD']*numwt
  else:
    noun_stuff[t[:-1]] = 0

print(noun_stuff)
feature3 = noun_stuff

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
{'official US President Barack Obama wants lawmakers weigh whether use military force Syria': 0.6, 'Obama sent letter heads House Senate Saturday night hours announcing believes military action Syrian targets right step take alleged use chemical weapons': 1.2, 'proposed legislation Obama asks Congress approve use military force deter disrupt prevent degrade potential future uses chemical weapons weapons mass destruction': 2.1, 'step set turn international crisis fierce domestic political battle': 0.8999999999999999, 'key questions looming debate UN weapons inspectors find Syria': 0.3, 'happens Congress votes': 0, 'Syrian government react': 0.6, 'televised address White House Rose Garden earlier Saturday president said would take case Congress wants': 0.6, 'believe authority carry military action without specific congressional a

In [ ]:
# # SIMILARITY TO SENTENCE WITH MAXIMUM TF_IDF
# sort_tfidf = sorted(feature1.items(), key=lambda x:x[1], reverse=True)
# max_sent = sort_tfidf[0][0]

# from sklearn.metrics.pairwise import cosine_similarity
# result = cosine_similarity(max_sent,text_tokens[0])
# feature4 = {}

In [ ]:
feature5 = {}

In [ ]:
sentence_score = dict()
for key, val in feature1.items():
  sentence_score[key] = feature1[key] + feature2[key] + feature3[key]

for key, val in sentence_score.items():
  print(f"{key} : {val}")
# sentence_score = feature1  # add other features when finished

official US President Barack Obama wants lawmakers weigh whether use military force Syria : 2.688677423714
Obama sent letter heads House Senate Saturday night hours announcing believes military action Syrian targets right step take alleged use chemical weapons : 1.8033859475897311
proposed legislation Obama asks Congress approve use military force deter disrupt prevent degrade potential future uses chemical weapons weapons mass destruction : 2.7201934725519394
step set turn international crisis fierce domestic political battle : 1.6254497781900923
key questions looming debate UN weapons inspectors find Syria : 0.9843791700646076
happens Congress votes : 1.070533553585089
Syrian government react : 1.5740307098506492
televised address White House Rose Garden earlier Saturday president said would take case Congress wants : 1.2318898139215786
believe authority carry military action without specific congressional authorization know country stronger take course actions even effective said : 

## Creating Sentence vectors

In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.8 MB/s 
     |████████████████████████████████| 3.8 MB 9.8 MB/s 
     |████████████████████████████████| 1.2 MB 40.8 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
     |████████████████████████████████| 6.5 MB 535 kB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=a068f923f5ffc037259d2bf80d58d7058d2500a84538e044a912dbdc8bbb4d00
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

sentence_embeddings = sbert_model.encode(text_tokens)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))
print('Sample BERT embedding vector', sentence_embeddings[0])

Sample BERT embedding vector - length 768
Sample BERT embedding vector [ 4.09313217e-02 -2.11688969e-02  2.57774303e-03 -2.24537291e-02
 -3.94022241e-02  1.54455733e-02  4.67091054e-02 -5.94240660e-03
  7.33520463e-02 -5.25787883e-02  9.08573437e-03 -4.97108102e-02
 -6.50105812e-03  2.00390778e-02  5.32323979e-02  4.61592078e-02
  1.33742094e-02 -5.60426749e-02 -2.56312825e-02 -1.96771766e-03
 -2.80238260e-02  2.99522243e-02 -1.62084308e-02 -2.18892582e-02
  5.21789007e-02  1.76040325e-02 -2.50307154e-02 -4.03809138e-02
  2.19644736e-02  3.59526575e-02 -7.19295815e-03 -1.53312590e-02
 -8.55844747e-03 -3.62769212e-03  1.13769727e-06  1.05644169e-03
  1.97449587e-02 -1.06623545e-02 -1.90549474e-02 -3.44956703e-02
  1.44523885e-02 -5.22952154e-02 -1.14130266e-02 -4.72464152e-02
 -7.87425712e-02  6.72810078e-02 -3.24749611e-02  1.81708019e-02
 -2.00736839e-02  4.53528501e-02 -4.94617643e-03 -2.09815539e-02
 -3.38425413e-02  1.37611628e-02 -5.78687377e-02  7.95154572e-02
 -2.89357477e-03  1

In [ ]:
len(sentence_embeddings)

76

## Clustering using self_organising maps

In [ ]:
!pip install sklearn-som

In [ ]:
#@title SOM SIZE
m =  3#@param {type:"integer"}
n = 4 #@param {type:"integer"}

In [ ]:
from sklearn_som.som import SOM
# m -> row len of the self organising map
# n -> col len of the self organising map
som = SOM(m, n, dim=768)

In [ ]:
# training the self organising maps
# here the sentence embedding size is n * dimension of data
som.fit(sentence_embeddings , epochs = 100)


In [ ]:
predictions = som.predict(sentence_embeddings)

In [ ]:
print(len(predictions))
print(set(predictions))
print(predictions[0])

76
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
10


In [ ]:
predictions

array([10, 10, 10,  4,  2,  4,  3,  8,  6,  4,  9,  5,  8,  3,  3, 11,  1,
        3,  2,  1,  1,  1,  0,  7, 11,  5,  6, 11,  6,  3,  6,  0,  7,  2,
        8,  9,  4,  0, 10, 10,  8,  2,  5,  2,  3,  8, 11,  6,  0,  9,  8,
        5, 11,  9, 11, 11,  2, 10,  5,  5,  3,  7,  3, 11,  7,  7, 11,  7,
        7,  8,  3,  1,  8,  1,  0,  7])

## Generating Summary

In [ ]:
# creating org_text -> prediction mapping
cluster_len = m*n
clusters = [[] for _ in range(cluster_len)]
for i , sentence in enumerate(text_tokens):
    clusters[predictions[i]].append(sentence)

clusters


[['needs time able analyze information samples Nesirky said',
  'world many dangers menace must confronted',
  'unclear',
  'quite concerned',
  'explanation offered discrepancy'],
 ['aim game mandate clear ascertain whether chemical weapons used UN spokesman Martin Nesirky told reporters Saturday',
  'British US intelligence reports say attack involved chemical weapons UN officials stressed importance waiting official report inspectors',
  'inspectors share findings UN SecretaryGeneral Ban Kimoon Ban said wants wait UN teams final report completed presenting UN Security Council',
  'Organization Prohibition Chemical Weapons nine inspectors belong said Saturday could take three weeks analyze evidence collected',
  'British intelligence put number people killed attack 350',
  'US Secretary State John Kerry Friday cited death toll 1429 400 children'],
 ['key questions looming debate UN weapons inspectors find Syria',
  'Top US officials said theres doubt Syrian government behind Syrian o

In [ ]:
summary = []

for cluster in clusters:
    summary.append(org_mapping[sorted(cluster,key=lambda x:sentence_score[x],reverse = True)[0]])

summary = sorted(summary,key = lambda x:pos_mapping[x])  # sorting to preserve order
summary = "\n".join(summary)
print(summary)
print("\n Original Text : \n")
print("\n".join(org_text))

It's a step that is set to turn an international crisis into a fierce domestic political battle.
But who used the weapons in the reported toxic gas attack in a Damascus suburb on August 21 has been a key point of global debate over the Syrian crisis.
It's unclear.
A senior administration official told CNN that Obama has the authority to act without Congress -- even if Congress rejects his request for authorization to use force.
"The two leaders agreed that the international community must deliver a resolute message to the Assad regime -- and others who would consider using chemical weapons -- that these crimes are unacceptable and those who violate this international norm will be held accountable by the world," the White House said.
"Our fear now is that the lack of action could embolden the regime and they repeat his attacks in a more serious way," said spokesman Louay Safi.
"Under the Constitution, the responsibility to declare war lies with Congress," the Republican lawmakers said.


In [ ]:
perc = (len(summary) / len("".join(org_text))) * 100
print(perc)

20.738118227657974




# Evaluation


It is required to check accuracy, relevance and usefulness of the summary created by an automatic summarizer
